In [23]:
import pandas as pd
import numpy as np
from time import time
import json
from sqlalchemy import create_engine
import re
import datetime

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches #for custom legends
import seaborn as sns
#from lifelines import KaplanMeierFitter #survival analysis library
#from lifelines.statistics import logrank_test #survival statistical testing

In [24]:
url='panamera-ods.cnsuxis6zqxr.us-west-2.redshift.amazonaws.com:5439/panameraods'
dbname = 'redshift'
credentials = json.load(open('C:\\Users\\OLX - Lucas Dresi\\Documents\\LicenciaGrid\\dbconfig.json'))[dbname]
engine = create_engine("postgresql://" + credentials['user'] + ":" + credentials['password'] + "@" + url)

## My account tap over DAU proxy

In [33]:
query = """

  SELECT date_event_nk ,country_sk ,  count(distinct case when trackevent NOT IN ('app_open', 'on_create', 'apps', 'push_dis','pushDis', 'push_rcv', 'pushRcv', 'UAReg', 'ua_reg', 'uareg','push_dismissed', 'push_show', 'fetching_matrix', 'not_found','permissions_impression','userBadgeCount','test_assignment','test_impression','test_discovered','FCMReg','APNSReg','appOpn','appOpen','google_play_services') AND trackevent IS NOT NULL then session_long else null end) as dau_proxy , 
                         count(distinct case when trackevent='myaccount_tap' then session_long else null end) as myaccount
        FROM ods.panameraolx_mea_hydra_ninja_android_last_month 
            WHERE date_event_nk > '2019-01-25'
             and app_version in ('13.08.02' , '13.08.01' , '13.08.00')
        GROUP BY 1,2


"""

In [34]:
df = pd.read_sql_query(query,engine)

In [35]:
df.head()

,date_event_nk,country_sk,dau_proxy,myaccount
0,2019-01-27,olx|mea|ug,1599,273
1,2019-01-26,olx|mea|ng,2445,585
2,2019-01-27,olx|mea|ng,2656,676
3,2019-01-26,olx|mea|tz,20,4
4,2019-01-27,olx|mea|gh,3594,661


In [37]:
df['myaccountuse'] = df['myaccount'] / df['dau_proxy'] 
df.groupby(['country_sk'])['myaccountuse'].mean()
df.head(20)

,date_event_nk,country_sk,dau_proxy,myaccount,myaccountuse
0,2019-01-27,olx|mea|ug,1599,273,0.170732
1,2019-01-26,olx|mea|ng,2445,585,0.239264
2,2019-01-27,olx|mea|ng,2656,676,0.254518
3,2019-01-26,olx|mea|tz,20,4,0.200000
4,2019-01-27,olx|mea|gh,3594,661,0.183918
5,2019-01-27,olx|mea|pk,82156,14631,0.178088
6,2019-01-27,olx|mea|tz,36,6,0.166667
7,2019-01-27,olx|mea|ke,7658,1139,0.148733
8,2019-01-26,olx|mea|ke,7371,1124,0.152489
9,2019-01-28,olx|mea|za,25029,3322,0.132726


## Funnel of my account tap to edit profile

In [5]:
# People which make login_show
query = """
WITH step1 as (
  SELECT t.date_event_nk,t.session_long,t.session_long_seq,t.time_event_utc 
        FROM ods.panameraolx_mea_hydra_ninja_android_last_month t
      WHERE trackevent='myaccount_tap'
      )
      
SELECT date_event_nk, COUNT(distinct session_long) FROM step1
        GROUP BY 1
        ORDER BY 1
"""

In [6]:
step1 = pd.read_sql_query(query,engine)

In [22]:
step1.head()

,date_event_nk,count
0,2019-01-15,1
1,2019-01-16,1
2,2019-01-18,34
3,2019-01-19,184
4,2019-01-20,319
5,2019-01-21,420
6,2019-01-22,4355
7,2019-01-23,8408
8,2019-01-24,14442
9,2019-01-25,16948


In [12]:
query = """
WITH step1 as (
  SELECT t.date_event_nk,t.session_long,t.session_long_seq,t.time_event_utc 
      FROM ods.panameraolx_mea_hydra_ninja_android_last_month t
      WHERE  trackevent='myaccount_tap'

          ),
    step2 as (
    
    SELECT a.date_event_nk,
           a.session_long,
           a.time_event_utc as tiime_step2 ,
           b.time_event_utc as tiime_step1
                FROM ods.panameraolx_mea_hydra_ninja_android_last_month a
                    INNER JOIN step1 b ON a.session_long=b.session_long AND a.date_event_nk=b.date_event_nk
                                                                        --AND a.session_long_seq=b.session_long_seq
                    WHERE trackevent='myaccount_tap_profile'
                    AND DATEDIFF(minute, tiime_step1 , tiime_step2) <  10
                    AND DATEDIFF(minute, tiime_step1 , tiime_step2) >=  0
             
    )
SELECT date_event_nk ,COUNT(distinct session_long) FROM step2
        GROUP BY 1
        ORDER BY 1
        
"""

In [13]:
step2 = pd.read_sql_query(query,engine)

In [14]:
funnel = pd.merge(step1,step2,how='left',on=['date_event_nk'] , validate = 'one_to_one')
funnel['Show_Complete'] = funnel['count_y'] / funnel['count_x']
funnel.drop(funnel.index[-1:], inplace=True)
funnel.head()

,date_event_nk,count_x,count_y,Show_Complete
0,2019-01-15,1,NaN,NaN
1,2019-01-16,1,NaN,NaN
2,2019-01-18,34,7.0,0.205882
3,2019-01-19,184,32.0,0.173913
4,2019-01-20,319,52.0,0.163009


In [15]:
print(funnel['count_y'].sum() / funnel['count_x'].sum())

0.1759699269451734


## View item to view profile

In [10]:
# People which make login_show
query = """
WITH step1 as (
  SELECT t.date_event_nk,t.session_long,t.session_long_seq,t.time_event_utc 
        FROM ods.panameraolx_mea_hydra_ninja_android_last_month t
      WHERE trackevent='view_item'
      )
      
SELECT date_event_nk, COUNT(distinct session_long) FROM step1
        GROUP BY 1
        ORDER BY 1
"""

In [11]:
step1 = pd.read_sql_query(query,engine)

In [12]:
step1.head()

,count
0,2518221


In [41]:
query = """
WITH step1 as (
  SELECT t.date_event_nk,t.session_long,t.session_long_seq,t.time_event_utc 
      FROM ods.panameraolx_mea_hydra_ninja_android_last_month t
      WHERE  trackevent='view_item'

          ),
    step2 as (
    
    SELECT a.date_event_nk,
           a.session_long,
           a.time_event_utc as tiime_step2 ,
           b.time_event_utc as tiime_step1
                FROM ods.panameraolx_mea_hydra_ninja_android_last_month a
                    INNER JOIN step1 b ON a.session_long=b.session_long AND a.date_event_nk=b.date_event_nk
                                                                        AND a.session_long_seq=b.session_long_seq
                    WHERE trackevent='social_view_listed'
                    AND DATEDIFF(minute, tiime_step1 , tiime_step2) <  10
                    AND DATEDIFF(minute, tiime_step1 , tiime_step2) >=  0
             
    )
SELECT  COUNT(distinct session_long) FROM step2
        
"""

In [42]:
step2 = pd.read_sql_query(query,engine)

In [43]:
funnel = pd.merge(step1,step2,how='left',on=['date_event_nk'] , validate = 'one_to_one')
funnel['Show_Complete'] = funnel['count_y'] / funnel['count_x']
funnel.drop(funnel.index[-1:], inplace=True)
funnel.head()

,date_event_nk,count_x,count_y,Show_Complete
0,2018-12-01,205853,30148,0.146454
1,2018-12-02,235684,35898,0.152314
2,2018-12-03,237001,35531,0.149919
3,2018-12-04,241768,36011,0.148949
4,2018-12-05,238418,35489,0.148852


In [44]:
print(funnel['count_y'].sum() / funnel['count_x'].sum())

0.1448839013577168


## My account use Android

### Edite profile

In [64]:
query = """
WITH step1 as (
  SELECT t.date_event_nk,t.session_long,t.session_long_seq,t.time_event_utc 
        FROM ods.panameraolx_mea_hydra_ninja_android_last_month t
      WHERE trackevent='myaccount_tap'
      and country_sk = 'olx|mea|pk'
      )
      
SELECT COUNT(distinct session_long) FROM step1
"""

In [65]:
step1 = pd.read_sql_query(query,engine)

In [66]:
step1.head()

,count
0,76799


In [67]:
query = """
WITH step1 as (
  SELECT t.date_event_nk,t.session_long,t.session_long_seq,t.time_event_utc 
      FROM ods.panameraolx_mea_hydra_ninja_android_last_month t
      WHERE  trackevent='myaccount_tap'
     and country_sk = 'olx|mea|pk'

          ),
    step2 as (
    
    SELECT a.date_event_nk,
           a.session_long,
           a.time_event_utc as tiime_step2 ,
           b.time_event_utc as tiime_step1
                FROM ods.panameraolx_mea_hydra_ninja_android_last_month a
                    INNER JOIN step1 b ON a.session_long=b.session_long AND a.date_event_nk=b.date_event_nk
                                                                        --AND a.session_long_seq=b.session_long_seq
                    WHERE trackevent='myaccount_tap_profile'
                    AND DATEDIFF(minute, tiime_step1 , tiime_step2) <  10
                    AND DATEDIFF(minute, tiime_step1 , tiime_step2) >=  0
                      and country_sk = 'olx|mea|pk'
             
    )
SELECT  COUNT(distinct session_long) FROM step2
        
"""

In [68]:
step2 = pd.read_sql_query(query,engine)

In [69]:
step2.head()

,count
0,17498


In [71]:
funnel_edit.head(20)

,date_event_nk,count_x,count_y,Show_Complete
0,2019-01-15,1,NaN,NaN
1,2019-01-16,1,NaN,NaN
2,2019-01-18,34,7.0,0.205882
3,2019-01-19,184,32.0,0.173913
4,2019-01-20,319,52.0,0.163009
5,2019-01-21,420,72.0,0.171429
6,2019-01-22,4355,883.0,0.202755
7,2019-01-23,8408,1577.0,0.187559
8,2019-01-24,14442,2645.0,0.183146
9,2019-01-25,16948,2915.0,0.171997


In [72]:
print(funnel_edit['count_y'].sum() / funnel_edit['count_x'].sum())

0.17084188911704312


In [73]:
edit_profile = step2 / step1

      count
0  0.227842


### Notifications

In [74]:
query = """
WITH step1 as (
  SELECT t.date_event_nk,t.session_long,t.session_long_seq,t.time_event_utc 
        FROM ods.panameraolx_mea_hydra_ninja_android_last_month t
      WHERE trackevent='myaccount_tap'
         and country_sk = 'olx|mea|pk'
      )
      
SELECT COUNT(distinct session_long) FROM step1

"""

In [75]:
step1 = pd.read_sql_query(query,engine)

In [76]:
step1.head()

,count
0,76799


In [77]:
query = """
WITH step1 as (
  SELECT t.date_event_nk,t.session_long,t.session_long_seq,t.time_event_utc 
      FROM ods.panameraolx_mea_hydra_ninja_android_last_month t
      WHERE  trackevent='myaccount_tap'  and country_sk = 'olx|mea|pk'

          ),
    step2 as (
    
    SELECT a.date_event_nk,
           a.session_long,
           a.time_event_utc as tiime_step2 ,
           b.time_event_utc as tiime_step1
                FROM ods.panameraolx_mea_hydra_ninja_android_last_month a
                    INNER JOIN step1 b ON a.session_long=b.session_long AND a.date_event_nk=b.date_event_nk
                                                                        AND a.session_long_seq=b.session_long_seq
                    WHERE trackevent='notifications_open'
                    AND DATEDIFF(minute, tiime_step1 , tiime_step2) <  10
                    AND DATEDIFF(minute, tiime_step1 , tiime_step2) >=  0
              and country_sk = 'olx|mea|pk'
    )
SELECT  COUNT(distinct session_long) FROM step2

        
"""

In [78]:
step2 = pd.read_sql_query(query,engine)

In [79]:
step2.head()

,count
0,28460


In [80]:
notification = step2 / step1

      count
0  0.370578


### My network

In [81]:
query = """
WITH step1 as (
  SELECT t.date_event_nk,t.session_long,t.session_long_seq,t.time_event_utc 
      FROM ods.panameraolx_mea_hydra_ninja_android_last_month t
      WHERE  trackevent='myaccount_tap'  and country_sk = 'olx|mea|pk'

          ),
    step2 as (
    
    SELECT a.date_event_nk,
           a.session_long,
           a.time_event_utc as tiime_step2 ,
           b.time_event_utc as tiime_step1
                FROM ods.panameraolx_mea_hydra_ninja_android_last_month a
                    INNER JOIN step1 b ON a.session_long=b.session_long AND a.date_event_nk=b.date_event_nk
                                                                        AND a.session_long_seq=b.session_long_seq
                    WHERE trackevent='myaccount_tap_mynetwork'
                    AND DATEDIFF(minute, tiime_step1 , tiime_step2) <  10
                    AND DATEDIFF(minute, tiime_step1 , tiime_step2) >=  0
                     and country_sk = 'olx|mea|pk'
             
    )
SELECT  COUNT(distinct session_long) FROM step2

        
"""

In [82]:
step2 = pd.read_sql_query(query,engine)

In [83]:
step2.head()

,count
0,5330


In [84]:
print(step2 / step1)

      count
0  0.069402


In [ ]:
mynetwork = step2 / step1

### Orders and billing

In [85]:
query = """
WITH step1 as (
  SELECT t.date_event_nk,t.session_long,t.session_long_seq,t.time_event_utc 
      FROM ods.panameraolx_mea_hydra_ninja_android_last_month t
      WHERE  trackevent='myaccount_tap'
       and country_sk = 'olx|mea|pk'

          ),
    step2 as (
    
    SELECT a.date_event_nk,
           a.session_long,
           a.time_event_utc as tiime_step2 ,
           b.time_event_utc as tiime_step1
                FROM ods.panameraolx_mea_hydra_ninja_android_last_month a
                    INNER JOIN step1 b ON a.session_long=b.session_long AND a.date_event_nk=b.date_event_nk
                                                                        AND a.session_long_seq=b.session_long_seq
                    WHERE trackevent='myaccount_monetization_order_billings_tap'
                    AND DATEDIFF(minute, tiime_step1 , tiime_step2) <  10
                    AND DATEDIFF(minute, tiime_step1 , tiime_step2) >=  0
                     and country_sk = 'olx|mea|pk'
             
    )
SELECT  COUNT(distinct session_long) FROM step2

        
"""

In [86]:
step2 = pd.read_sql_query(query,engine)

In [87]:
step2.head()

,count
0,5980


In [88]:
print(step2 / step1)

      count
0  0.077866


In [ ]:
orders = step2 / step1

### Settings

In [89]:
query = """
WITH step1 as (
  SELECT t.date_event_nk,t.session_long,t.session_long_seq,t.time_event_utc 
      FROM ods.panameraolx_mea_hydra_ninja_android_last_month t
      WHERE  trackevent='myaccount_tap'
       and country_sk = 'olx|mea|pk'

          ),
    step2 as (
    
    SELECT a.date_event_nk,
           a.session_long,
           a.time_event_utc as tiime_step2 ,
           b.time_event_utc as tiime_step1
                FROM ods.panameraolx_mea_hydra_ninja_android_last_month a
                    INNER JOIN step1 b ON a.session_long=b.session_long AND a.date_event_nk=b.date_event_nk
                                                                        AND a.session_long_seq=b.session_long_seq
                    WHERE trackevent='myaccount_tap_settings'
                    AND DATEDIFF(minute, tiime_step1 , tiime_step2) <  10
                    AND DATEDIFF(minute, tiime_step1 , tiime_step2) >=  0
                     and country_sk = 'olx|mea|pk'
             
    )
SELECT  COUNT(distinct session_long) FROM step2

        
"""

In [90]:
step2 = pd.read_sql_query(query,engine)

In [91]:
step2.head()

,count
0,12166


In [92]:
print(step2 / step1)

      count
0  0.158414


In [ ]:
settings = step2 / step1 

### Help

In [95]:
query = """
WITH step1 as (
  SELECT t.date_event_nk,t.session_long,t.session_long_seq,t.time_event_utc 
      FROM ods.panameraolx_mea_hydra_ninja_android_last_month t
      WHERE  trackevent='myaccount_tap'
       and country_sk = 'olx|mea|pk'

          ),
    step2 as (
    
    SELECT a.date_event_nk,
           a.session_long,
           a.time_event_utc as tiime_step2 ,
           b.time_event_utc as tiime_step1
                FROM ods.panameraolx_mea_hydra_ninja_android_last_month a
                    INNER JOIN step1 b ON a.session_long=b.session_long AND a.date_event_nk=b.date_event_nk
                                                                        AND a.session_long_seq=b.session_long_seq
                    WHERE trackevent='myaccount_tap_help'
                    AND DATEDIFF(minute, tiime_step1 , tiime_step2) <  10
                    AND DATEDIFF(minute, tiime_step1 , tiime_step2) >=  0
                     and country_sk = 'olx|mea|pk'
             
    )
SELECT  COUNT(distinct session_long) FROM step2

        
"""

In [96]:
step2 = pd.read_sql_query(query,engine)

In [97]:
step2.head()

,count
0,3620


In [98]:
print(step2 / step1)

      count
0  0.047136


In [99]:
helpp = step2 / step1 

In [ ]:
d = {'col1': [1, 2], 'col2': [3, 4]}
df = pd.DataFrame(data=d)
df
